<a href="https://colab.research.google.com/github/GarlandZhang/cs486_project/blob/main/hashtag_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Standard imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import math
import keras
import keras.backend as K
from keras import Model
from keras.models import load_model
from keras.layers import Embedding, Dot, Add
from keras.activations import sigmoid
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.applications.vgg19 import VGG19
import os
from PIL import Image
import cv2
import pickle
from IPython.display import clear_output

Collaborative Filtering

In [ ]:
# project_path = './'
project_path = '/content/drive/My Drive/AI(CS486)'
IMG_DIM = 512
IMG_CHANNELS = 3

feature_size = 1000

hashtags = []
with open(os.path.join(project_path, 'data', 'hashtags.txt')) as f:
  for line in f:
    hashtags.append(line)

num_imgs = 2936 # n: number of images 
num_hashtags = len(hashtags) # m

inputs = np.load(os.path.join(project_path, 'data', 'train_inputs.npy')).tolist()

inputs_map = {}
for input in inputs:
  if inputs_map.get(input[0]) == None:
    inputs_map[input[0]] = { input[1] : True }
  else:
    inputs_map[input[0]][input[1]] = True

# labels
labels = [1 for i in range(len(inputs))]

for i in range(num_imgs):
  for j in range(num_hashtags):
    if inputs_map.get(i) == None:
      labels.append(0)
      inputs.append([i, j])
      inputs_map[i] = { j : True }
    elif inputs_map[i].get(j) == None: # false label
      labels.append(0)
      inputs.append([i, j])
      inputs_map[i][j] = True
      
labels = np.array(labels, dtype=np.float32)

inputs = np.array(inputs)

In [ ]:
batch_sizes = [256]
# batch_sizes = [32, 64]
# feature_sizes = [1000, 10000]
feature_sizes = [1000]
num_epochs = 100
init_lrs = [1e-2, 1e-3, 1e-4]
# init_lrs = [1e-2, 1e-3]

In [ ]:
class HashtagPredictor(Model):
  def __init__(self, num_imgs, num_hashtags, feature_size, **kwargs):
    super(HashtagPredictor, self).__init__(**kwargs)
    self.num_imgs = num_imgs + 1 # + 1 for test image ..?
    self.num_hashtags = num_hashtags
    self.feature_size = feature_size
    self.img_embedding = Embedding(self.num_imgs, self.feature_size, embeddings_initializer="he_normal", )
    self.img_bias = Embedding(self.num_imgs, 1)
    self.hashtag_embedding = Embedding(self.num_hashtags, self.feature_size, embeddings_initializer="he_normal")
    self.hashtag_bias = Embedding(self.num_hashtags, 1)

  def call(self, inputs):
    img_matrix = self.img_embedding(inputs[:, 0])
    img_bias = self.img_bias(inputs[:, 0])
    hashtag_matrix = self.hashtag_embedding(inputs[:, 1])
    hashtag_bias = self.hashtag_bias(inputs[:, 1])
    raw_preds = Dot(axes=1)([img_matrix, hashtag_matrix])
    preds_with_bias = Add()([raw_preds, img_bias, hashtag_bias])
    preds = sigmoid(raw_preds)
    return preds

In [ ]:
for b, batch_size in enumerate(batch_sizes):
  for lr_ind, init_lr in enumerate(init_lrs):
    for feat_ind, feature_size in enumerate(feature_sizes):
        print(f'Creating model with batch size: {batch_size}, learning rate: {init_lr}, feature size: {feature_size}')
        model = HashtagPredictor(num_imgs, num_hashtags, feature_size)
        model.compile(loss='binary_crossentropy', optimizer=Adam(lr=init_lr), metrics=['accuracy'])

        history = model.fit(
            x=inputs,
            y=labels,
            batch_size=batch_size,
            epochs=num_epochs,
            verbose=1,
            validation_split=0.2
        )
        model.save(os.path.join(project_path, 'models', f'model_bs_{batch_size}_lr_{init_lr}_fs_{feature_size}'))

        with open(os.path.join(project_path, 'histories', f'history_bs_{batch_size}_lr_{init_lr}_fs_{feature_size}'), 'wb') as f:
          pickle.dump(history.history, f)
        
        clear_output()